In [ ]:
import pptk
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 1. Test on Random Data

In [ ]:
points=torch.randn([1000, 3]).numpy()
color = torch.randn([1000, 3])
color = color - color.min()
print(color.min(), color.max())
color = color / color.max() / 1.001
color = color.numpy()
color.max()

In [ ]:
skip = 1  # Skip every n points

fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection='3d')
point_range = range(0, points.shape[0], skip) # skip points to prevent crash
ax.scatter(points[point_range, 0],   # x
           points[point_range, 1],   # y
           points[point_range, 2],   # z
           facecolors=color, # height data for color
           cmap='Spectral',
           marker="o")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.axis('auto')  # {equal, scaled}
ax.view_init(15, 15)
plt.show()

# 2. Read & Render Local Mesh

In [ ]:
ply_path="/data/pkurei/mpeg/dataset/longdress/Ply/longdress_vox10_1051.ply"

In [ ]:
import sys, os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [ ]:
from mpeg_process import read_mesh

In [ ]:
mesh = read_mesh(ply_path)

In [ ]:
color, pos = mesh.color.numpy(), mesh.pos.numpy()

In [ ]:
from mpl_toolkits.mplot3d import proj3d
import matplotlib as mpl
#Make sure these are floating point values:                                                                                                                                                                                              
x_scale = pos[:, 0].max()
y_scale = pos[:, 1].max()
z_scale = pos[:, 2].max()

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0
print(scale)

mpl.rcParams['legend.fontsize'] = 10

def get_subplot_proj(ax):
    def short_proj():
        return np.dot(Axes3D.get_proj(ax), scale)

    return short_proj

In [ ]:
cm = 1/2.54
fig = plt.figure(figsize=(120 * cm, 60 * cm))
# fig.subplots_adjust(bottom=-0.15,top=1.2, left=0.1, right=0.1)
fig.subplots_adjust(bottom=-0.2,top=1.2, left=-0.2, right=1.2)
ax = fig.add_subplot(121, projection='3d')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
# ax.set_title('Sample Point Cloud')



ax.get_proj=get_subplot_proj(ax)
ax.scatter(pos[:,0],   # x
           pos[:,1],   # y
           pos[:,2],   # z
           facecolors=color/255., s=0.1# height data for color
          )
ax.view_init(90, -90)
ax = fig.add_subplot(122, projection='3d')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')


ax.get_proj=get_subplot_proj(ax)
ax.scatter(pos[:,0],   # x
           pos[:,1],   # y
           pos[:,2],   # z
           facecolors=color/255., s=0.1# height data for color
          )
ax.view_init(90, -90)
plt.show()
fig.savefig('test.png', dpi=300)

# 3. Test Trained Model

In [ ]:
%load_ext autoreload
%autoreload
from visualize import get_subplot_proj, visualize
from utils import load_model, get_model, process_whole
from bf import BilateralFilterv2
from dataloader import normal_noise
import torch
import torch_geometric as tgnn
import pretty_errors
# import importlib
# importlib.reload('utils')

In [ ]:
parallel=False
model = get_model(
    dataset_type='MPEG',
    bfilter=BilateralFilterv2,
    batch_size=24,
    device=torch.device('cpu'),
    parallel=parallel,
    reg=0.0,
    loss_type='mse'
)

In [ ]:
load_model(
    model=model.module if parallel else model,
    optimizer=None,
    f='../model/mpeg-bf-10.0v3sgd+reg+act/271/model-latest.save',
    optim=None,
    e=0,
    evaluate=None,
)

In [ ]:
reconstructed, noisy, mesh, mse_error = process_whole(
    model,
    ply_path,
    noise_generator=normal_noise,
    sigma=10.0,
    batch_size=24,
)